<a href="https://colab.research.google.com/github/93gh4ucf/Coursera_Capstone/blob/main/Agrupaci%C3%B3n_y_Segmentaci%C3%B3n_de_Vecindarios_en_la_Ciudad_de_Nueva_York_(part_III).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [5]:
#!conda install -c conda-forge geopy --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas

# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas

print('Libraries imported.')


Libraries imported.


**Web Scrapin de los Datos**

Por medio de BeatifulSoup se hace Web Scraping para obtener los datos de la página web que contiene los códigos postales de Toronto.



In [6]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")


Una vez hecha la sopa de datos se traen solo los datos de la tabla ya que contienen los codigos postales de Toronto.

In [7]:
table = soup.find('table')

In [8]:
file=[]
ncol=list(range(0,9))

for i in ncol :

    for row in table.find_all('tr'): # in html table row is represented by the table
    # Get all columns in each row.
        cols = row.find_all('p') # in html a column is represented by the tag td or p
        file.append(cols[i].getText())

    # convert to dataframe:
df= pd.DataFrame(file)

**Tabla de Códigos Postales**

Despues de traer los datos que estan contenidos en la tabla, por medio de pandas se le da formato a la tabla para obtener la salida deseada.

In [9]:
df["Postal Code"]=df[0].str[0:3]
df["Borough"]=df[0].str[3:].str.split("(",n=1,expand=True)[0]
df["Neigh"]=df[0].str.rsplit("(",n=2,expand=True)[1].str.split(")",n=1,expand=True)[0]
df["Neighbourhood"]=df["Neigh"].str.replace(" / ",",")

In [10]:
postalcode_list=df[(df["Borough"]!="Not assigned\n")]
postalcode_list=postalcode_list.drop([0,"Neigh"],axis=1)

In [11]:
postalcode_list.head()

,Postal Code,Borough,Neighbourhood
1,M1B,Scarborough,"Malvern,Rouge"
2,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [9]:
postalcode_list.shape


(103, 3)

**Cordenadas de Latitud y Longitud**

Por medio del paquete Geocoder Python: https://geocoder.readthedocs.io/index.html. Se obtienen los datos de Latitud y Longitud para cada código postal.

Para asegurarse de obtener las coordenadas de todos los vecindarios, se puede ejecutar un ciclo while para cada código postal.

In [12]:
#"http://cocl.us/Geospatial_data"
Geospatial_Coordinates=pd.read_csv("/content/Geospatial_Coordinates.csv")

Una vez obtenidas las coordenadas se adjuntan al dataset de códigos postales



In [13]:
coord_list=pd.merge(postalcode_list,Geospatial_Coordinates,on="Postal Code")

In [14]:
coord_list.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**Agrupación y Segmentación**

A continuación vamos a empezar a utilizar el API de FourSquare para explorar los barrios y segmentarlos.

Mapa de los barrios de Toronto
Se va a generar un mapa de Toronto con todos sus barrios, para iniciar el análisis de segmentación

Utilizando la libreria geopy, se obtiene las coordenadas de Toronto. Para poder definir una instancia del geocoder necesitaremos definir un user_agent. Nombraremos a nuestro agente ny_explorer, como se muestra a continuación.

In [16]:
address = 'TORONTO,CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
Para el ejecicio de eligirán los barrios que contienen la palabra Toronto

In [17]:
toronto_data = coord_list[coord_list['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.groupby("Borough").size().sort_values(ascending=False)

Borough
Downtown Toronto                                                17
Central Toronto                                                  9
West Toronto                                                     6
East Toronto                                                     4
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                            1
dtype: int64

Se ubican los barrios de Toronto en un mapa

In [18]:
# crear un mapa de Manhattan usando los valores de latitud y longitud
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# añadir los marcadores al mapa
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

Make this Notebook Trusted to load map: File -> Trust Notebook